In [92]:
import spacy
import os
import pandas as pd
import geopy 
import matplotlib.pyplot as plt
import requests
from bs4 import BeautifulSoup #Parses HTML content
from collections import Counter #Counts item frequencies in a list
from geopy.extra.rate_limiter import RateLimiter
from spacy import displacy

### Using URL

In [94]:
url = "https://www.cnn.com/travel/article/experts-guide-to-great-american-road-trips/index.html"
data = requests.get(url)

In [95]:
#Parse out HTML tags and clean up newlines
soup = BeautifulSoup(data.content, 'html.parser')
text = soup.get_text()
text = text.strip().replace("\n", " ").replace("\r", " ")

In [96]:
# requires conda install -c conda-forge spacy-model-en_core_web_lg

nlp = spacy.load("en_core_web_lg")
locations = []
doc = nlp(text)


OSError: [E050] Can't find model 'en_core_web_lg'. It doesn't seem to be a Python package or a valid path to a data directory.

In [6]:
displacy.render(doc, style="ent")

In [27]:
locations = []
locations.extend([[ent.text, ent.start, ent.end, ent.label_, spacy.explain(ent.label_)] for ent in doc.ents ])
df_new = pd.DataFrame(locations, columns=['Location', 'start','end', 'label', 'meaning'])
df_new

,Location,start,end,label,meaning
0,American,2,3,NORP,Nationalities or religious or political groups
1,American,19,20,NORP,Nationalities or religious or political groups
2,CNN •,25,27,ORG,"Companies, agencies, institutions, etc."
3,12th August,28,30,DATE,Absolute or relative dates or periods
4,Rome,45,46,GPE,"Countries, cities, states"
...,...,...,...,...,...
140,Cable News Network,1785,1788,ORG,"Companies, agencies, institutions, etc."
141,Warner Media Company,1790,1793,ORG,"Companies, agencies, institutions, etc."
142,CNN,1798,1799,ORG,"Companies, agencies, institutions, etc."
143,™ & ©,1800,1803,ORG,"Companies, agencies, institutions, etc."


In [ ]:
meanings = []
df_new['meaning'] = df_new['label'].apply(spacy.explain())

In [29]:
look_for = ['GPE']

locations = []
locations.extend([[ent.text, ent.start, ent.end, ent.label_] for ent in doc.ents if ent.label_ in look_for])
 
df = pd.DataFrame(locations, columns=['Location', 'start','end', 'label'])
df

,Location,start,end,label
0,Rome,45,46,GPE
1,China,70,71,GPE
2,Kerouac,148,149,GPE
3,U.S.,189,190,GPE
4,California,192,193,GPE
5,Highway,194,195,GPE
6,Connecticut,259,260,GPE
7,Florida,261,262,GPE
8,U.S.,357,358,GPE
9,Atlanta,434,435,GPE


In [91]:
locator = geopy.geocoders.Nominatim(user_agent='mygeocoder')
geocode = RateLimiter(locator.geocode, min_delay_seconds=1)

# df.loc[df['Location']=='the Empire State Building', 'Location'] = 'Empire State Building'
# df

### need to apply some clean up like removing 'the' and capitalizing

df['address'] = df['Location'].apply(geocode)

In [8]:
df

,Location,start,end,label
0,Rome,45,46,GPE
1,China,70,71,GPE
2,Kerouac,148,149,GPE
3,U.S.,189,190,GPE
4,California,192,193,GPE
5,Highway,194,195,GPE
6,Route 66,198,200,FAC
7,New England,202,204,LOC
8,Connecticut,259,260,GPE
9,Florida,261,262,GPE


In [21]:
df['coordinates'] = df['address'].apply(lambda loc: tuple(loc.point) if loc else None)

df[['latitude', 'longitude', 'altitude']] = pd.DataFrame(df['coordinates'].tolist(), index=df.index)

df.latitude.isnull().sum()

df = df[pd.notnull(df['latitude'])]

In [22]:
df

,Location,start,end,label,address,coordinates,latitude,longitude,altitude
0,Rome,45,46,GPE,"(Roma, Roma Capitale, Lazio, 00100, Italia, (4...","(41.8933203, 12.4829321, 0.0)",41.893320,12.482932,0.0
1,China,70,71,GPE,"(中国, (35.000074, 104.999927))","(35.000074, 104.999927, 0.0)",35.000074,104.999927,0.0
2,Kerouac,148,149,GPE,"(Kerouac, Rosporden, Quimper, Finistère, Breta...","(47.9850734, -3.8102109, 0.0)",47.985073,-3.810211,0.0
3,U.S.,189,190,GPE,"(Unterer See, Bahnhof, Böblingen (Stadt), Böbl...","(48.6840674, 9.009582476304603, 0.0)",48.684067,9.009582,0.0
4,California,192,193,GPE,"(California, United States, (36.7014631, -118....","(36.7014631, -118.755997, 0.0)",36.701463,-118.755997,0.0
...,...,...,...,...,...,...,...,...,...
115,US,1196,1197,GPE,"(United States, (39.7837304, -100.445882))","(39.7837304, -100.445882, 0.0)",39.783730,-100.445882,0.0
116,Route 66,1319,1321,FAC,"(Route 66, Anaheim, Orange County, California,...","(33.8058796, -117.9189832, 0.0)",33.805880,-117.918983,0.0
117,California,1651,1652,GPE,"(California, United States, (36.7014631, -118....","(36.7014631, -118.755997, 0.0)",36.701463,-118.755997,0.0
118,Washington,1653,1654,GPE,"(Washington, District of Columbia, United Stat...","(38.8950368, -77.0365427, 0.0)",38.895037,-77.036543,0.0


In [30]:
import wikipedia


In [31]:
locations = list(df['Location'])
locations

['Rome',
 'China',
 'Kerouac',
 'U.S.',
 'California',
 'Highway',
 'Connecticut',
 'Florida',
 'U.S.',
 'Atlanta',
 'Georgia',
 'New York',
 'Redondo Beach',
 'California',
 'America',
 'U.S.',
 'Winslow',
 'Arizona',
 'U.S.',
 'China',
 'the United States',
 'US',
 'California',
 'Cawker City',
 'Kansas',
 'Darwin',
 'Minnesota',
 'Collinsville',
 'Illinois',
 'America',
 'Cabazon',
 'California',
 'Katoosa',
 'Oklahoma',
 'Vermont',
 'West Virginia',
 'New Jersey',
 'buffalo',
 'Jamestown',
 'North Dakota',
 'Rapid City',
 'America',
 'Arizona',
 'California',
 'Texas',
 'Nebraska',
 'US',
 'US',
 'US',
 'California',
 'Washington',
 'D.C.']

In [11]:
from geotext import GeoText

In [89]:

cities = []
states = []
us_places = {}

vetted_locs = []

for loc in locations[:15]:
    print("Starting Location:")
    print(loc)
    try:

        summary = str(wikipedia.summary(loc))
        print("1 try")
    except:
        alts = wikipedia.search(loc)
        for x in range(len(alts)):
            try:   
                summary = wikipedia.summary(wikipedia.search(loc)[x])
                print("2 try")
                break
            except:
                summary = ''
    if len(summary) > 0:    
        ner_texts = []
        ner_labels = []
        
        doc = nlp(summary)
        # displacy.render(doc, style="ent")
        
        ner_texts.extend([ent.text.lower() for ent in doc.ents])
        ner_labels.extend([[ent.label_] for ent in doc.ents])

        # check if original text was tagged and is a place
        if loc.lower() in ner_texts:
            # print(ner_texts)

            loc_idx = ner_texts.index(loc.lower())
            matching_label = ner_labels[loc_idx][0]

            if (matching_label == 'GPE') or (matching_label == 'LOC'):
                print(f"Vetted location: {loc}")

                places = GeoText(summary)
                cities_counter = Counter(places.cities)
                print("Found these cities in wiki:")
                print(cities_counter)
                print("\n")

            else:
                print(matching_label)
                print("Untrustworthy entity. Token is not a place. Skipping.\n")
        else:
            print("Untrustworthy entity. Token not in text. Skipping.\n")

        # temp_locations.extend([[ent.text, ent.start, ent.end, ent.label_] for ent in doc.ents if ent.label_ in ['LOC', 'GPE', 'FAC']])
        # print(locations)
        # us_places[loc] = {'city':'', 
        # 'state':''}
        # if 'city' in summary:
        #     us_places[loc]['city'] = summary
        # if 'state' in summary:
        #     us_places[loc]['state'] = summary


    else:
        print("no result")

Starting Location:
Rome
1 try
Untrustworthy entity. Token not in text. Skipping.

Starting Location:
China
1 try
Vetted location: China
Found these cities in wiki:
Counter({'Macau': 1, 'Beijing': 1, 'Asia': 1, 'Summit': 1})


Starting Location:
Kerouac
1 try
Vetted location: Kerouac
Found these cities in wiki:
Counter({'March': 1, 'Lowell': 1, 'New York': 1})


Starting Location:
U.S.
1 try
Vetted location: U.S.
Found these cities in wiki:
Counter({'Washington': 1, 'New York': 1, 'Union': 1})


Starting Location:
California
1 try
Vetted location: California
Found these cities in wiki:
Counter({'Los Angeles': 4, 'New York': 4, 'San Francisco': 3, 'Oregon': 1, 'Sacramento': 1, 'San Bernardino': 1, 'Hollywood': 1, 'Douglas': 1})


Starting Location:
Highway
1 try
Untrustworthy entity. Token not in text. Skipping.

Starting Location:
Connecticut
1 try
Vetted location: Connecticut
Found these cities in wiki:
Counter({'New York': 3, 'Long Island': 2, 'Hartford': 2, 'New Haven': 2, 'Bridgepor

In [90]:
import geopy

In [84]:
wikipedia.summary('Atlanta')

'The Atlantic Ocean is the second-largest of the world\'s five oceans, with an area of about 106,460,000 km2 (41,100,000 sq mi). It covers approximately 20% of Earth\'s surface and about 29% of its water surface area. It is known to separate the "Old World" of Africa, Europe and Asia from the "New World" of the Americas in the European perception of the World.\nThe Atlantic Ocean occupies an elongated, S-shaped basin extending longitudinally between Europe and Africa to the east, and the Americas to the west. As one component of the interconnected World Ocean, it is connected in the north to the Arctic Ocean, to the Pacific Ocean in the southwest, the Indian Ocean in the southeast, and the Southern Ocean in the south (other definitions describe the Atlantic as extending southward to Antarctica). The Atlantic Ocean is divided in two parts, by the Equatorial Counter Current, with the North(ern) Atlantic Ocean and the South(ern) Atlantic Ocean at about 8°N.Scientific explorations of the A